In [1]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [2]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/german_dataset/german_dataset.tsv',sep='\t', encoding="utf-8")
df_train=df_train[df_train.task_2!='NONE']

In [3]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_2'],test_size=0.15,random_state=140) 

In [4]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [5]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/german_dataset/hasoc_de_test_gold.tsv',sep='\t', encoding="utf-8") 
df_test=df_test[df_test.task_2!='NONE']
test_y = encoder.fit_transform(df_test['task_2']) #encoding the label of testing data

In [6]:
# create a tfidf vectorizer object 
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=25999)
tfidf_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_tfidf =  tfidf_vect.transform(train_x)  #get tfidf vector features for training data
xvalid_tfidf =  tfidf_vect.transform(valid_x)  #get tfidf vector features for validation data
x_tfidf =  tfidf_vect.transform(df_test['text'])  #get tfidf vector features for testing data

In [7]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [8]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc(),x_tfidf.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.29      0.31      0.30        13
           1       0.75      0.56      0.64        43
           2       0.25      0.67      0.36         6

    accuracy                           0.52        62
   macro avg       0.43      0.51      0.43        62
weighted avg       0.60      0.52      0.54        62

classification report for testing
              precision    recall  f1-score   support

           0       0.15      0.32      0.20        19
           1       0.84      0.58      0.69       112
           2       0.06      0.20      0.09         5

    accuracy                           0.53       136
   macro avg       0.35      0.37      0.32       136
weighted avg       0.72      0.53      0.60       136

